# 3DGS模型渲染
需要先完成3DGS模型训练

In [6]:
import os
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from random import randint
from gaussian_renderer import render, network_gui 
import sys
from tqdm import tqdm

from scene import Scene
from os import makedirs
from gaussian_renderer import render
import torchvision
from utils.general_utils import safe_state
from argparse import ArgumentParser
from arguments import ModelParams, PipelineParams, get_combined_args
from gaussian_renderer import GaussianModel

from utils.general_utils import load_config

In [ ]:
def render_set(model_path, name, iteration, views, gaussians, pipeline, background):
    render_path = os.path.join(model_path, name, "ours_{}".format(iteration), "renders")
    gts_path = os.path.join(model_path, name, "ours_{}".format(iteration), "gt")
 
    makedirs(render_path, exist_ok=True)
    makedirs(gts_path, exist_ok=True)
 
    for idx, view in enumerate(tqdm(views, desc="Rendering progress")):
        rendering = render(view, gaussians, pipeline, background)["render"]
        gt = view.original_image[0:3, :, :]
        original_filename = os.path.basename(view.image_name)
 
        #mip360
        # torchvision.utils.save_image(rendering, os.path.join(render_path, original_filename + ".JPG"))
        # torchvision.utils.save_image(gt, os.path.join(gts_path, original_filename + ".JPG"))
 
 
        #blender 保存图片
        torchvision.utils.save_image(rendering, os.path.join(render_path, original_filename + ".png"))
        torchvision.utils.save_image(gt, os.path.join(gts_path, original_filename + ".png"))
 

def render_sets(dataset : ModelParams, iteration : int, pipeline : PipelineParams, skip_train : bool, skip_test : bool):
    with torch.no_grad():
        gaussians = GaussianModel(dataset.sh_degree)
        scene = Scene(dataset, gaussians, load_iteration=iteration, shuffle=False)
 
        bg_color = [1,1,1] if dataset.white_background else [0, 0, 0]
        background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")
 
        if not skip_train:
             render_set(dataset.model_path, "train", scene.loaded_iter, scene.getTrainCameras(), gaussians, pipeline, background)
 
        if not skip_test:
             render_set(dataset.model_path, "test", scene.loaded_iter, scene.getTestCameras(), gaussians, pipeline, background)

In [5]:
# Set up command line argument parser
sys.argv[1:] = [
    "-m", "../OUTPUTS/3DGS/ship",
    "--skip_train", 
    "--config", "configs/blender.yml"
]
parser = ArgumentParser(description="Testing script parameters")
model = ModelParams(parser, sentinel=True)
pipeline = PipelineParams(parser)
parser.add_argument("--iteration", default=-1, type=int)
parser.add_argument("--skip_train", action="store_true")
parser.add_argument("--skip_test", action="store_true")
parser.add_argument("--quiet", action="store_true")
parser.add_argument("--config", type=str, required=True, help="Path to configuration YAML file")
args = get_combined_args(parser)
print("Rendering " + args.model_path)

args = load_config(args)
# args.source_path = args.gt_source_path

# Initialize system state (RNG)
safe_state(args.quiet)

render_sets(model.extract(args), args.iteration, pipeline.extract(args), args.skip_train, args.skip_test)

Looking for config file in ../OUTPUTS/3DGS/ship/cfg_args
Config file found: ../OUTPUTS/3DGS/ship/cfg_args
Rendering ../OUTPUTS/3DGS/ship
source_path: ../OUTPUTS/HR/ship

hr_source_path: ../NERF_SYNTHETIC/ship

lr_source_path: ../OUTPUTS/LR/ship

gt_source_path: ../NERF_SYNTHETIC/ship

Loading trained model at iteration 30000 [05/11 15:29:23]
Found transforms_train.json file, assuming Blender data set! [05/11 15:29:23]
Reading Training Transforms [05/11 15:29:23]
Reading Test Transforms [05/11 15:29:31]
Loading Training Cameras [05/11 15:29:52]
Loading Test Cameras [05/11 15:29:53]


Rendering progress: 100%|██████████| 200/200 [00:54<00:00,  3.69it/s]


Using device: cuda [05/11 15:37:20]
